This notebook presents my work for the **Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto** as part of **Applied Data Science Capstone**.

**Section 1**

Import needed libraries.

In [1]:
import pandas as pd
import urllib.request
import html5lib
import numpy as np
from bs4 import BeautifulSoup

Get html content from the wikipedia page.

In [2]:
p_can_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
fp = urllib.request.urlopen(p_can_url)
p_can_html = fp.read()
p_can_html = p_can_html.decode("utf8")
fp.close()
p_can = BeautifulSoup(p_can_html)

F:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file F:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Get the postcode and neighbourhood table from the html content, and delete the rows where Borough equals Not assigned.

In [3]:
df_p_can = []
for tr in p_can.table.find_all('tr'):
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    df_p_can.append(row)
df_p_can=pd.DataFrame(df_p_can,columns=['Postcode','Borough','Neighbourhood'])
df_p_can=df_p_can[df_p_can['Borough']!='Not assigned']
df_p_can=df_p_can[df_p_can.Borough.notnull()]
df_p_can['Neighbourhood']=df_p_can['Neighbourhood'].replace({'\n':''}, regex=True)
df_p_can.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


If Neighbourhood is Not assigned, name it with the name of Borough.

In [4]:
df_p_can['Neighbourhood'] = np.where(df_p_can['Neighbourhood'] == 'Not assigned', df_p_can['Borough'], df_p_can['Neighbourhood'])

Combine Neighbourhoods with the same Borough into one cell, and split using commas.

In [5]:
df_p_can=df_p_can.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

View table and the size of the table.

In [37]:
df_p_can.head(11)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [38]:
df_p_can.shape

(103, 3)

**Section 2**

Use the csv file to create the following dataframe:

In [7]:
coord = pd.read_csv('http://cocl.us/Geospatial_data')

In [8]:
df_coord_can=pd.merge(df_p_can, coord, left_on='Postcode', right_on='Postal Code').drop(['Postal Code'], axis=1)
df_coord_can.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


**Section 3**

Extract only postal codes from Toronto.

In [9]:
import folium

In [10]:
df_t=df_coord_can[df_coord_can['Borough'].isin(['Downtown Toronto','East Toronto','West Toronto','Central Toronto'])]
df_t.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Visualize all neighbourhoods in Toronto on a map.

In [11]:
map_t = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_t['Latitude'], df_t['Longitude'], df_t['Borough'], df_t['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_t)  
    
map_t

Define API credential.

In [14]:
CLIENT_ID = 'x'
CLIENT_SECRET = 'x'
VERSION = '20180605' # Foursquare API version

Define a function to get venues by coordinates input and use Toronto neighbourhoods' data as input to get venues near by the neighbourhoods.

In [17]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_venues = getNearbyVenues(names=df_t['Neighbourhood'],
                                   latitudes=df_t['Latitude'],
                                   longitudes=df_t['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [19]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Check how many venues each neighbourhoods have.

In [20]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,59,59,59,59,59,59
"Brockton,Exhibition Place,Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",13,13,13,13,13,13
"Cabbagetown,St. James Town",44,44,44,44,44,44
Central Bay Street,78,78,78,78,78,78
"Chinatown,Grange Park,Kensington Market",98,98,98,98,98,98
Christie,15,15,15,15,15,15


Analyse what are the top 10 venues for each neighbourhoods and store the information in a dataframe.

In [21]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Bar,Thai Restaurant,Café,Steakhouse,Hotel,American Restaurant,Bakery,Sushi Restaurant,Asian Restaurant
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Pub,Farmers Market,Cheese Shop,Seafood Restaurant,Italian Restaurant,Bakery,Steakhouse
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Gym,Climbing Gym,Falafel Restaurant,Convenience Store,Burrito Place,Stadium,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Farmers Market,Garden,Skate Park,Brewery,Comic Shop,Fast Food Restaurant,Spa,Burrito Place,Butcher
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden


Import Kmeans for clustering.

In [25]:
from sklearn.cluster import KMeans

Cluster the neighbourhoods using k means. And add the cluster label back to the Toronto neighbourhood data set.

In [26]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [28]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_t_c=df_t.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

Visualize the clusters on map.

In [29]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [30]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_t_c['Latitude'], df_t_c['Longitude'], df_t_c['Neighbourhood'], df_t_c['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Finally, check the neighbourhoods in each clusters.

In [31]:
df_t_c.loc[df_t_c['Cluster Labels'] == 0, df_t_c.columns[[1] + list(range(5, df_t_c.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Health Food Store,Coffee Shop,Pub,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
41,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Yoga Studio,Brewery,Bubble Tea Shop,Café,Restaurant
42,East Toronto,0,Movie Theater,Intersection,Ice Cream Shop,Pub,Sandwich Place,Fish & Chips Shop,Sushi Restaurant,Steakhouse,Fast Food Restaurant,Coffee Shop
43,East Toronto,0,Café,Coffee Shop,Italian Restaurant,Bakery,Gastropub,American Restaurant,Fish Market,Juice Bar,Bookstore,Latin American Restaurant
45,Central Toronto,0,Hotel,Breakfast Spot,Dance Studio,Food & Drink Shop,Burger Joint,Sandwich Place,Park,Gym,Gym / Fitness Center,Event Space
46,Central Toronto,0,Sporting Goods Shop,Coffee Shop,Clothing Store,Fast Food Restaurant,Salon / Barbershop,Diner,Mexican Restaurant,Dessert Shop,Rental Car Location,Furniture / Home Store
47,Central Toronto,0,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Restaurant,Café,Sushi Restaurant,Coffee Shop,Dance Studio,Deli / Bodega
49,Central Toronto,0,Coffee Shop,Pub,American Restaurant,Light Rail Station,Sports Bar,Medical Center,Supermarket,Sushi Restaurant,Bagel Shop,Fried Chicken Joint
51,Downtown Toronto,0,Coffee Shop,Restaurant,Park,Pizza Place,Bakery,Italian Restaurant,Café,Market,Pub,Breakfast Spot
52,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Burger Joint,Café,Bubble Tea Shop,Gastropub,Men's Store,Sushi Restaurant


In [32]:
df_t_c.loc[df_t_c['Cluster Labels'] == 1, df_t_c.columns[[1] + list(range(5, df_t_c.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Playground,Tennis Court,Women's Store,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [33]:
df_t_c.loc[df_t_c['Cluster Labels'] == 2, df_t_c.columns[[1] + list(range(5, df_t_c.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,2,Park,Playground,Trail,Women's Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
64,Central Toronto,2,Park,Trail,Sushi Restaurant,Jewelry Store,Women's Store,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [34]:
df_t_c.loc[df_t_c['Cluster Labels'] == 3, df_t_c.columns[[1] + list(range(5, df_t_c.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3,Home Service,Garden,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [35]:
df_t_c.loc[df_t_c['Cluster Labels'] == 4, df_t_c.columns[[1] + list(range(5, df_t_c.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4,Park,Swim School,Bus Line,Women's Store,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
